In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

In [6]:

df = pd.read_csv('Allocation/SubParts/Sabaragamuwa.csv')

In [7]:
# Get the columns as x and y values
X = df.drop(columns=['Landslide'])
y = df['Landslide']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Imputer for non-numeric columns
non_numeric_imputer = SimpleImputer(strategy='most_frequent')
X_train_non_numeric_imputed = non_numeric_imputer.fit_transform(X_train.select_dtypes(exclude=['number']))
X_test_non_numeric_imputed = non_numeric_imputer.transform(X_test.select_dtypes(exclude=['number']))

# Imputer for numeric columns
numeric_imputer = SimpleImputer(strategy='mean')
X_train_numeric_imputed = numeric_imputer.fit_transform(X_train.select_dtypes(include=['number']))
X_test_numeric_imputed = numeric_imputer.transform(X_test.select_dtypes(include=['number']))

# Concatenate the imputed data back together
X_train_imputed = pd.concat([pd.DataFrame(X_train_numeric_imputed, columns=X_train.select_dtypes(include=['number']).columns),
                            pd.DataFrame(X_train_non_numeric_imputed, columns=X_train.select_dtypes(exclude=['number']).columns)],
                            axis=1)

X_test_imputed = pd.concat([pd.DataFrame(X_test_numeric_imputed, columns=X_test.select_dtypes(include=['number']).columns),
                            pd.DataFrame(X_test_non_numeric_imputed, columns=X_test.select_dtypes(exclude=['number']).columns)],
                            axis=1)


X_train_imputed.to_csv('imputed_trainData.csv',index=False)
X_test_imputed.to_csv('imputed_test_data.csv',index=False)

In [10]:
label_encoder = LabelEncoder()
X['Month'] = label_encoder.fit_transform(X['Month'])
X['Serial']=label_encoder.fit_transform(X['Serial'])






In [11]:
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train_imputed, y_train)


# Make predictions on the test set
y_pred = model.predict(X_test_imputed)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

ValueError: could not convert string to float: 'LA2007'